In [305]:
import numpy as np
import pandas as pd
import sys
import datetime as dt
from datetime import date
from time import mktime
import requests
from bs4 import BeautifulSoup
from typing import List
import re
import io
from dateutil.relativedelta import relativedelta

In [245]:
lista_indices = ["%5EIBEX", "%5EBFX","%5EBVSP", "%5EDJI", "%5EFCHI", "%5EFTSE", "%5EGDAXI", "%5EHSI", 
                "%5EMXX", "%5EJKSE", "%5EMERV", "%5EOMXSPI", "%5EOSEAX", "%5ESSMI", "%5ESTI"]

In [45]:
def convert_to_unix(date):
    """
    converts date to unix timestamp
    
    parameters: date - in format (dd-mm-yyyy)
    
    returns integer unix timestamp
    """
    datum = dt.datetime.strptime(date, '%d-%m-%Y')
    
    return int(mktime(datum.timetuple()))


In [49]:
def get_crumbs_and_cookies(ticker: str):
    # Thanks to MAIK ROSENHEINRICH
    """
    get crumb and cookies for historical data csv download from yahoo finance  
    parameters: stock - short-handle identifier of the company    
    returns a tuple of header, crumb and cookie
    """   
    url = 'https://finance.yahoo.com/quote/{}/history'.format(ticker)
    
    with requests.session():
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
                   }        
        website = requests.get(url, headers=header)
        soup = BeautifulSoup(website.text, 'lxml')
        
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))
        output=(header, crumb[0], website.cookies)
        return output   


In [47]:
day_begin_unix = convert_to_unix("01-12-2018")
day_end_unix = convert_to_unix("21-09-2020")

In [50]:
header, crumb, cookies = get_crumbs_and_cookies(lista_indices[1])

In [70]:
with requests.session():
        
            url = 'https://query1.finance.yahoo.com/v7/finance/download/' \
                '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}' \
                .format(stock=lista_indices[1], day_begin=day_begin_unix, day_end=day_end_unix, interval='1d', crumb=crumb)
                
            website = requests.get(url, headers=header, cookies=cookies)

In [377]:
def get_historical_data(tickers: List[str], day_begin: str, day_end: str, interval='1d'):

    historical_prices = None
    df_create = False


    for ticker in tickers:
        error1='404 Not Found: Timestamp data missing.' 
    
        day_begin_unix = convert_to_unix(day_begin)
        day_end_unix = convert_to_unix(day_end)   
    
        header, crumb, cookies = get_crumbs_and_cookies(ticker)
    
        with requests.session():
        
            url = 'https://query1.finance.yahoo.com/v7/finance/download/' \
                '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}' \
                .format(stock=ticker, day_begin=day_begin_unix, day_end=day_end_unix, interval=interval, crumb=crumb)

            print(url)
                
            website = requests.get(url, headers=header, cookies=cookies)
            if website.status_code == 200:
                if not df_create:
                    historical_prices = pd.read_csv(io.StringIO(website.text))
                    historical_prices["ticker"] = ticker
                    df_create = True
                else:
                    temp_df = pd.read_csv(io.StringIO(website.text))
                    temp_df["ticker"] = ticker
                    historical_prices = historical_prices.append(temp_df, ignore_index=True)
                
                print("Obteniendo histórico para {}".format(ticker))
                print("longitud del df: {}".format(len(historical_prices.Date)))
            
            else:
                print("Error downloading data from ticker {}".format(ticker))
                print("Response was {}".format(website.text))


    historical_prices.drop_duplicates()
    historical_prices.Date = historical_prices.Date.astype(np.datetime64)
    #historical_prices = historical_prices.dropna()
    return historical_prices 

In [146]:
def get_currency(tickers: List[str]):
    
    indice_divisa = []

    for ticker in tickers:

        header, crumb, cookies = get_crumbs_and_cookies(ticker)
            
        with requests.session():
                
            url = "https://es.finance.yahoo.com/quote/"+ticker+"/components/"            
            website = requests.get(url, headers=header, cookies=cookies)

        soup = BeautifulSoup(website.text)
        divisa = re.findall('Divisa en [a-zA-Z]{3}', str(soup))
        divisa = divisa[0][len(divisa[0])-3:]
        indice_divisa.append((ticker, divisa))

    return indice_divisa




In [206]:
def get_components(tickers=List[str]):

    componentes = {}

    for ticker in tickers:

        header, crumb, cookies = get_crumbs_and_cookies(ticker)
                    
        with requests.session():
                        
            intentos = 3
            while intentos > 0:
                url = "https://es.finance.yahoo.com/quote/"+ticker+"/components/"            
                website = requests.get(url, headers=header, cookies=cookies)

                print("Obteniendo componentes de {}".format(ticker))
                if website.status_code == 200:

                    soup = BeautifulSoup(website.text)
                    try:
                        df = pd.read_html(str(soup))[0]
                        df = df.iloc[:,[0,1]]
                        componentes[ticker] = df.to_dict()

                    except ValueError as ve:
                        print(ve)
                        print(url)
                        next
                    intentos = 0
                else:
                    print("Error al conectar con yahoo, intentos restantes: {}".format(intentos))
                    intentos -= 1
                    if intentos == 0:
                        print("No se ha conseguido descargar información para el ticker: {}".format(ticker))


        
    return componentes



In [281]:
def get_market_days(lista_indices: List[str], historical_prices: pd.DataFrame):
    market_days = {}
    for idx in lista_indices:
        market_days[idx] = list(historical_prices[historical_prices["ticker"] == idx].Date)

    return market_days

In [330]:
def get_business_days(begin_date: str, end_date: str):

    b = dt.datetime.strptime(begin_date, "%d-%m-%Y")
    e = dt.datetime.strptime(end_date, "%d-%m-%Y")

    bdays = pd.date_range(start=b, end=e, freq='B')
    bdays = bdays.to_frame(index=False)
    bdays.columns = ['Date']

    return bdays

In [396]:
def homogenize(price_data: pd.DataFrame, dates: pd.DataFrame):

    indices = np.unique(price_data["ticker"])
    homogenized_prices = []

    for idx in indices:
        tmp_df = pd.merge(price_data[price_data.ticker == idx], dates, how="outer", on="Date")

        tmp_df.sort_values(by='Date', ascending=True, inplace=True)
        
        # Sustituir NA por el valor de la fila anterior. Si la fila con NA es la 1ª deja el NA sin dar un error.
        print(tmp_df.isna().any())
        tmp_df.fillna(method='ffill', inplace=True)
        print(tmp_df.isna().any())
        # Si los NA están en las primeras filas no 
        # hemos solucionado el problema. En principio no debería de haber más que dos (sábado y domingo), 
        # pero una empresa podría no cotizar lunes, martes... por lo que no conocemos el nº de potenciales NA a resolver.
        # Para resolver el problema, invertimos el orden del DF, aplicamos na.locf de nuevo y devolvemos el DF a su posición original.
        tmp_df.sort_values(by='Date', ascending=True, inplace=True)
        tmp_df.fillna(method='ffill', inplace=True)
        print(tmp_df.isna().any())
        tmp_df.sort_values(by='Date', ascending=False, inplace=True)
        break 
    
        homogenized_prices.append(tmp_df)

    
    return homogenized_prices

In [397]:
x = homogenize(df, dates)


        Date         Open         High          Low        Close    Adj Close  \
0 2018-01-02  3990.370117  3991.899902  3957.760010  3979.530029  3979.530029   
1 2018-01-03  3988.100098  4012.330078  3974.209961  4005.610107  4005.610107   
2 2018-01-04  4025.780029  4065.600098  4023.330078  4057.919922  4057.919922   
3 2018-01-05  4067.739990  4097.939941  4067.699951  4097.040039  4097.040039   
4 2018-01-08  4114.399902  4124.459961  4110.500000  4114.529785  4114.529785   

       Volume  ticker  
0  21160100.0  %5EBFX  
1  25869300.0  %5EBFX  
2  32912700.0  %5EBFX  
3  25470800.0  %5EBFX  
4  21194600.0  %5EBFX  
          Date         Open         High          Low        Close  \
510 2018-01-01          NaN          NaN          NaN          NaN   
0   2018-01-02  3990.370117  3991.899902  3957.760010  3979.530029   
1   2018-01-03  3988.100098  4012.330078  3974.209961  4005.610107   
2   2018-01-04  4025.780029  4065.600098  4023.330078  4057.919922   
3   2018-01-05  406

In [332]:
dates = get_business_days("01-01-2018", "01-01-2019")


In [335]:
tmp = df[df.ticker == np.unique(df["ticker"])[0]]

In [378]:
# Descargamos los indices de la lista
df = get_historical_data(lista_indices, "01-01-2018", "01-01-2020", interval='1d')
# Para cada indice listamos los componentes que cotizan en el indice
componentes = get_components(lista_indices)
# Nos quedamos solo con los indices de los que tenemos componentes
indices = list(set(componentes.keys()) & set(np.unique(df["ticker"])))
df = df[df.ticker.isin(indices)]
# Extraemos los días con los que queremos trabajar ( puede haber días que el índice no cotizó pero queremos tener los precios de ese día)
market_days = get_business_days("01-01-2018", "01-01-2020")
# Homogeneizamos los datos de los indices







https://query1.finance.yahoo.com/v7/finance/download/%5EIBEX?period1=1514761200&period2=1577833200&interval=1d&events=history&crumb=wP2sZn2Cviu
Obteniendo histórico para %5EIBEX
longitud del df: 511
https://query1.finance.yahoo.com/v7/finance/download/%5EBFX?period1=1514761200&period2=1577833200&interval=1d&events=history&crumb=vbV42MqwWF\u002F
Obteniendo histórico para %5EBFX
longitud del df: 1021
https://query1.finance.yahoo.com/v7/finance/download/%5EBVSP?period1=1514761200&period2=1577833200&interval=1d&events=history&crumb=4Sg5z7OCJ.N
Obteniendo histórico para %5EBVSP
longitud del df: 1515
https://query1.finance.yahoo.com/v7/finance/download/%5EDJI?period1=1514761200&period2=1577833200&interval=1d&events=history&crumb=RSvNnLubdwK
Obteniendo histórico para %5EDJI
longitud del df: 2018
https://query1.finance.yahoo.com/v7/finance/download/%5EFCHI?period1=1514761200&period2=1577833200&interval=1d&events=history&crumb=nmcjPaZR.7X
Obteniendo histórico para %5EFCHI
longitud del df: 2529
